In [ ]:
# import packages
from __future__ import print_function


import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual,IntSlider
import ipywidgets as widgets
from IPython.core.display import display, HTML

#import plotly.offline as py
#autosize =False
import plotly.io as pio


In [ ]:
death_df     = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv") 
confirmed_df = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv") 
recovered_df = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv")
df           = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")


In [ ]:
df.columns = map(str.lower, df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)


In [ ]:
del df['sno']

In [ ]:
df           = df.rename(columns           = {"province/state":"state","country/region": "country",})
confirmed_df = confirmed_df.rename(columns = {"province/state": "state", "country/region": "country"})
recovered_df = recovered_df.rename(columns = {"province/state": "state", "country/region": "country"})
death_df     = death_df.rename(columns     = {"province/state": "state", "country/region": "country"})


Based on the latest updates, I have colored confirmed cases, recovered cases and deaths.

In [ ]:
nan_states_df = df[df['state'].isnull()]

print('nan_states_df shape is : '+ str(nan_states_df.shape))
print('nan_states_df has got : '+ str(nan_states_df['country'].nunique()) + ' unique Country values')

nan_states_df = nan_states_df[['observationdate','country','confirmed','deaths','recovered']]
nan_states_df.head()

In [ ]:
states_df = df[df['state'].notnull()]

print('states_df shape is : '+ str(states_df.shape))
print('states_df has got : '+ str(states_df['state'].nunique()) + ' unique State values')
 
states_df = states_df[['observationdate','state','country','confirmed','deaths','recovered']]
states_df.head()

> I have divided the countries in the data with states(states_df) and without a 

---

state(nan_states_df).
  
    
# Concentrating and preparing dataframes

In [ ]:
concentrated_states_df= states_df.groupby(['observationdate','country'])[['confirmed','deaths','recovered']].sum().reset_index()
concentrated_states_df.head()

In [ ]:
full_countries_df = pd.concat([nan_states_df, concentrated_states_df], axis=0).reset_index()
full_countries_df.head()

In [ ]:
lastest_full_countries_df = full_countries_df.groupby(['country'])[['observationdate','confirmed','deaths','recovered']].max().reset_index()
lastest_full_countries_df.head()

In [ ]:
sorted_countries_df = lastest_full_countries_df.sort_values("confirmed", ascending = False)
sorted_countries_df.head()

In [ ]:
# sorting the values by confirmed descednding order
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    temp_df = pd.DataFrame('', index=x.index, columns=x.columns)
    temp_df.iloc[:, 2] = y
    temp_df.iloc[:, 3] = r
    temp_df.iloc[:, 4] = g
    
    return temp_df


def show_latest_cases(n):
    n = int(n)
    return sorted_countries_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10');

## USA

In [ ]:
usa_df = states_df[states_df['country']=='US'] 
usa_df.head()

In [ ]:
lastest_usa_df = usa_df.groupby(['state']).max().reset_index()
lastest_usa_df.head()

# Exploring Current Status in the USA

In [ ]:
confirmed_usa_total = int(lastest_usa_df['confirmed'].sum())
deaths_usa_total = int(lastest_usa_df['deaths'].sum())
recovered_usa_total = int(lastest_usa_df['recovered'].sum())

In [ ]:
# displaying the USA total stats

display(HTML("<div style = 'background-color: white; padding: 30px '>" +
             "<span style='color: black; font-size:30px;'> Confirmed: "  + str(confirmed_usa_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_usa_total) + "</span>"+
             "<span style='color: darkgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_usa_total) + "</span>"+
             "</div>")
       )

### What are the top 10 states with the most cases?

In [ ]:
sorted_lastest_usa_df = lastest_usa_df.sort_values(by='confirmed', ascending=False)
sorted_lastest_usa_df[:10]

### Comparison of States in the USA

In [ ]:
f, ax = plt.subplots(figsize=(10,15))

sns.set_color_codes("bright")
sns.barplot(x="confirmed", y="state", data=sorted_lastest_usa_df[:50],
            label="Confirmed", color="y")

sns.set_color_codes("bright")
sns.barplot(x="recovered", y="state", data=sorted_lastest_usa_df[:50],
            label="Recovered", color="g")

sns.set_color_codes("bright")
sns.barplot(x="deaths", y="state", data=sorted_lastest_usa_df[:50],
            label="Deaths", color="r")

# Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 400), ylabel="",
       xlabel="Stats")
sns.despine(left=True, bottom=True)

In [ ]:
fig_pie = px.pie(lastest_usa_df, values = 'confirmed',names='state', height=600)
fig_pie.update_traces(textposition='inside', textinfo='percent+label')

fig_pie.update_layout(
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

pio.show(fig_pie)

## Are there states where all of the cases have died?

In [ ]:
lastest_usa_df[(lastest_usa_df['confirmed'] == lastest_usa_df['deaths'])]

## Are there any states where there are more deaths than survivors?

In [ ]:
lastest_usa_df[(lastest_usa_df['recovered'] < lastest_usa_df['deaths'])]

## Deaths and Recovered ratio in confirmed cases

In [ ]:
lastest_usa_df['death ratio'] = (lastest_usa_df['deaths']/lastest_usa_df['confirmed'])*100

In [ ]:
lastest_usa_df['recovered ratio'] = (lastest_usa_df['recovered']/lastest_usa_df['confirmed'])*100

In [ ]:
lastest_usa_df.sort_values(by='death ratio', ascending=False).head()

 > According to these results, Grant County, WA and Santa Rosa County, FL are the cities with the highest mortality rates.
 
 >However, the number of dead is quite low, so I think I should set a limit

In [ ]:
lastest_usa_df[lastest_usa_df.confirmed>100].sort_values(by='death ratio', ascending=False).head()

 > When I make the number of confirmed more than 100, I see that the state with the highest death rate is Connecticut. 

In [ ]:
lastest_usa_df.sort_values(by='recovered ratio', ascending=False).head()

Also according this table, again I should put a limit to confirmed cases

In [ ]:
lastest_usa_df[lastest_usa_df.confirmed>100].sort_values(by='recovered ratio', ascending=False).head()

When I look at the results from this table, Michigan has the best recovered rate.

## Let's look at the distribution of cases and deaths to America in a tree map

In [ ]:
fig_c = px.treemap(lastest_usa_df, path=["state"], values="confirmed", 
                 title='Number of Confirmed Cases in the USA',
                 color_discrete_sequence = px.colors.qualitative.Pastel)
fig_c.show()

In [ ]:
fig_d = px.treemap(lastest_usa_df, path=["state"], values="deaths", 
                 title='Number of Deaths Cases in the USA',
                 color_discrete_sequence = px.colors.qualitative.Pastel)
fig_d.show()

## Other countries in the world

In [ ]:
confirmed_total = int(sorted_countries_df['confirmed'].sum())
deaths_total = int(sorted_countries_df['deaths'].sum())
recovered_total = int(sorted_countries_df['recovered'].sum())
active_total = confirmed_total-deaths_total-recovered_total

In [ ]:
# displaying the total stats

display(HTML("<div style = 'background-color: white; padding: 30px '>" +
             "<span style='color: black; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: darkgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [ ]:
# # plotting the 10 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_countries_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10);

>* Enter the name of your country(in capitalized format(e.g. Italy)) and world for total cases


In [ ]:
# I will use plotly again,
import plotly.graph_objects as go

def plot_cases_for_country(country):
    labels = ["confirmed", "deaths"]
    colors = ["blue", "red"]
    mode_size = [6,8]
    line_size = [4,5]

    df_list = [confirmed_df, death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == "World" or country == "world":
            x_data = np.array(list(df.iloc[:,5:])) # after 5th column we have dates on datasets
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis = 0)
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df["country"]== country].iloc[:,5:]),axis = 0)
        fig.add_trace(go.Scatter(x=x_data, y= y_data, mode="lines+markers", 
                                name = labels[i],
                                line = dict(color = colors[i], width = line_size[i]),
                                connectgaps = True,
                                text = "Total" + str(labels[i]) + " : " + str(y_data[-1])
                                                    ))
       # pio.show(fig)
       #py.iplot(fig)
        fig.show()
       
        
# so make it interactive
interact(plot_cases_for_country, country = "World");

In [ ]:
 import folium

In [ ]:
world_map = folium.Map(location=[11,0], titles = "cartodbpositron", zoom_start = 2, max_zoom = 6, min_zoom = 2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location = [confirmed_df.iloc[i]["lat"],confirmed_df.iloc[i]["long"]],
    fill= True,
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*25000,
    fill_color = "blue",
    color = "darkorange",
    tooltip = "<div style='margin:0; background-color: grey; color:white;'>" +
                "<h4 style= 'text-align:center; font-weight:bold'>"+confirmed_df.iloc[i]["country"]+"</h4>"
                "<hr style='margin:10px;color:white;'>"+
                "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                    "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                    "<li>Deaths: "+str(death_df.iloc[i,-1])+"</li>"+
                    "<li>Deaeth Rate: "+str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+"</li>"+
                "</ul></div>"
    ).add_to(world_map)

world_map

# Exploring Cases Over Time Worldwide with Interactive Time Map

## Spread of Coronavirus

In [ ]:
fig1 = px.choropleth(full_countries_df, 
                    locations="country", 
                    locationmode = "country names",
                    color="confirmed",
                    color_continuous_scale='sunsetdark',
                    hover_name="country", 
                    animation_frame="observationdate"
                   )

fig1.update_layout(
    title_text = 'Spread of Coronavirus',
    title_x = 0.85,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig1.show()

## Spread of Recovered from Corona Virus

In [ ]:
fig2 = px.choropleth(full_countries_df, 
                    locations="country", 
                    locationmode = "country names",
                    color="recovered",
                    #color_continuous_scale='Rainbow',
                    hover_name="country", 
                    animation_frame="observationdate"
                   )

fig2.update_layout(
    title_text = 'Number of Corona Virus recovered',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig2.show()